# Proofs
In this jupyter notebook, proofs for the required conditions set in the problem description and follow-up emails are provided.

Names:
    **Bellamkonda Sri Krishna Chaitanya** &
    **Bellamkonda Aaditya Sri Krishna**


In [1]:
# Imports
import matplotlib.pyplot as plt
import heapq
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from math import floor, ceil, inf
from dataclasses import asdict

from edinburgh_challenge.constants import police_stations, police_stations_dict
from edinburgh_challenge.utility import generate_early_shift_distributions
from edinburgh_challenge.models import NaiveModel, GreedyModel, EnhancedModel, SimplifiedModelNotBest, SimplifiedModel
from edinburgh_challenge.simulation import *
from edinburgh_challenge.processing import calculate_metric, calculate_simulation_performance

In [2]:
source = "./data.xlsx"
data = pd.read_excel(source)
data["Time"] = (data["Day"]-1)*24 + data["Hour"]
data.columns = [x.lower() for x in data.columns]

In [3]:
ps = [[f'PS_{i}', p.x, p.y] for i,p in 
      enumerate([police_stations.one, 
                 police_stations.two, 
                 police_stations.three])]
df_ps = pd.DataFrame(ps, columns=["Name","Latitude", "Longitude"])

# Running the best Model

In [4]:
# Running the best model
shift_distribution = {'Early': {'Station_1': 0, 'Station_2': 5, 'Station_3': 10},
  'Day': {'Station_1': 2, 'Station_2': 19, 'Station_3': 4},
  'Night': {'Station_1': 9, 'Station_2': 3, 'Station_3': 28}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]

simulation = SimulationWithMaxUtilisation(data, ps_coords, shift_distribution, 
                        verbose=-1)



greedy_model = GreedyModel(shift_distribution, police_stations_dict)

# Proofs

### Proof that number of officers per shift was not exceeded

In [5]:
# No. of officers per shift
simulation.run(greedy_model)
print(f"Performance: {calculate_simulation_performance(simulation.analyze_simulation_results())}")

day=1 shift='Early' no_of_officers=15
day=1 shift='Day' no_of_officers=25
day=1 shift='Night' no_of_officers=40
day=2 shift='Early' no_of_officers=15
day=2 shift='Day' no_of_officers=25
day=2 shift='Night' no_of_officers=40
day=3 shift='Early' no_of_officers=15
day=3 shift='Day' no_of_officers=25
day=3 shift='Night' no_of_officers=40
day=4 shift='Early' no_of_officers=15
day=4 shift='Day' no_of_officers=25
day=4 shift='Night' no_of_officers=40
day=5 shift='Early' no_of_officers=15
day=5 shift='Day' no_of_officers=25
day=5 shift='Night' no_of_officers=40
day=6 shift='Early' no_of_officers=15
day=6 shift='Day' no_of_officers=25
day=6 shift='Night' no_of_officers=40
day=7 shift='Early' no_of_officers=15
day=7 shift='Day' no_of_officers=25
day=7 shift='Night' no_of_officers=40
Performance: 0.9973702067085207


In [6]:
cumulative_insidents = simulation.cumulative_incidents

filtered_incidents = []
for incident in cumulative_insidents:
    filtered_incident = {
        'urn': incident.urn,
        'day': incident.day,
        'hour': incident.hour,
        'priority': incident.priority,
        'resolving_officer': incident.resolving_officer,
        'response_time': incident.response_time, 
        'allocation_time': incident.allocation_time
    }
    filtered_incidents.append(filtered_incident)

No. of officers utilised for for cases in a particular day and shift.

In [7]:
# Modified script to ensure output for all seven days of the week
from collections import defaultdict

# Defining the get_shift function
def get_shift(hour_in_day):
    if 0 <= hour_in_day < 8:
        return "Early"
    elif 8 <= hour_in_day < 16:
        return "Day"
    else:
        return "Night"

# Initialize counts for all shifts for each day of the week
officer_count_per_day_and_shift = {day: {"Early": set(), "Day": set(), "Night": set()} for day in range(7)}

# Populate the counts based on the data
for incident in cumulative_insidents:
    time = incident.allocation_time 
    day_of_week = time // 24  # Calculating the day of the week
    hour_in_day = time % 24   # Hour within the day
    if "Officer_Station_3_Early_2" in incident.resolving_officer:
        shift = get_shift(hour_in_day)
        if shift == "Day":
            print(time, day_of_week, hour_in_day)

    shift = get_shift(hour_in_day)

    officer_count_per_day_and_shift[day_of_week][shift].add(incident.resolving_officer)

# Convert sets to counts and format the output as a string for each day
output_strings = []
for day in range(7):
    shifts = officer_count_per_day_and_shift[day]
    output_strings.append(
        f"Day {day + 1}: Early: {len(shifts['Early'])}, Day: {len(shifts['Day'])}, Night: {len(shifts['Night'])}"
    )

output_strings

['Day 1: Early: 15, Day: 25, Night: 40',
 'Day 2: Early: 15, Day: 25, Night: 40',
 'Day 3: Early: 15, Day: 25, Night: 39',
 'Day 4: Early: 15, Day: 25, Night: 40',
 'Day 5: Early: 15, Day: 25, Night: 40',
 'Day 6: Early: 15, Day: 25, Night: 40',
 'Day 7: Early: 15, Day: 25, Night: 40']

### Proof that an officer was sent to each incident

In [8]:
# Display the filtered incidents
pd.DataFrame(filtered_incidents)

,urn,day,hour,priority,resolving_officer,response_time,allocation_time
0,PS-20220706-0009,1,0,Prompt,Officer_Station_2_Early_3,0.318837,0.000000
1,PS-20220706-0021,1,0,Prompt,Officer_Station_3_Early_1,0.200332,0.000000
2,PS-20220706-0028,1,0,Prompt,Officer_Station_3_Early_0,1.155379,1.065088
3,PS-20220706-0035,1,0,Prompt,Officer_Station_3_Early_0,0.215088,0.000000
4,PS-20220706-0043,1,0,Prompt,Officer_Station_2_Early_3,1.061780,0.918837
...,...,...,...,...,...,...,...
2267,PS-20220715-3261,7,23,Prompt,Officer_Station_3_Night_17,167.093501,167.000000
2268,PS-20220715-3270,7,23,Immediate,Officer_Station_1_Night_1,167.156019,167.000000
2269,PS-20220715-3276,7,23,Prompt,Officer_Station_3_Night_3,167.175542,167.000000
2270,PS-20220715-3279,7,23,Immediate,Officer_Station_1_Night_2,167.094645,167.000000


In [9]:
officer_count_per_day_and_shift[2]["Night"]

{'Officer_Station_1_Night_0',
 'Officer_Station_1_Night_1',
 'Officer_Station_1_Night_2',
 'Officer_Station_1_Night_3',
 'Officer_Station_1_Night_4',
 'Officer_Station_1_Night_5',
 'Officer_Station_1_Night_6',
 'Officer_Station_1_Night_7',
 'Officer_Station_1_Night_8',
 'Officer_Station_2_Night_0',
 'Officer_Station_2_Night_1',
 'Officer_Station_2_Night_2',
 'Officer_Station_3_Night_0',
 'Officer_Station_3_Night_1',
 'Officer_Station_3_Night_10',
 'Officer_Station_3_Night_11',
 'Officer_Station_3_Night_12',
 'Officer_Station_3_Night_13',
 'Officer_Station_3_Night_14',
 'Officer_Station_3_Night_15',
 'Officer_Station_3_Night_16',
 'Officer_Station_3_Night_17',
 'Officer_Station_3_Night_18',
 'Officer_Station_3_Night_19',
 'Officer_Station_3_Night_2',
 'Officer_Station_3_Night_20',
 'Officer_Station_3_Night_21',
 'Officer_Station_3_Night_22',
 'Officer_Station_3_Night_23',
 'Officer_Station_3_Night_24',
 'Officer_Station_3_Night_25',
 'Officer_Station_3_Night_26',
 'Officer_Station_3_Nig

# Time Travel

Initialization of Officer Assignments:
It initializes officer_assignments, a dictionary to track the incidents resolved by each officer.
The assignments are based on shift_distribution, which presumably contains the distribution of officers across various shifts and stations.
For each shift and station, it creates officer names in the format "Officer_{station}_{shift}_{i}" and initializes their assignment list as empty.

Setting Up Incident Response Tracking:
incident_response is a dictionary used to track the total number of incidents and the number of incidents resolved within a target time, categorized by their priority levels: 'Immediate', 'Prompt', and 'Standard'.

Flag for Time Travel Detection:
time_travel_occurred is a boolean flag initially set to False. It's used to detect any anomalies in incident resolution times that might suggest 'time travel' (i.e., a later incident being resolved before an earlier one).

Processing Resolved Incidents:
The method iterates over resolved_incidents from simulation.
For each resolved incident, it performs several checks and updates:
Officer Assignment Update: If an officer is assigned to the incident, their resolution time is added to their assignment list in officer_assignments. If the officer resolves another incident at an earlier time than a previous one, time_travel_occurred is set to True.
Incident Response Tracking: The total count of incidents and the count of incidents resolved within the target time are updated in incident_response based on the incident's priority.
Target Time Calculation: The target time for resolution is determined based on the priority of the incident, with different times for 'Immediate', 'Prompt', and 'Standard' categories.

Calculating Percentages:
The method concludes with a loop to calculate percentages based on the data in incident_response. However, the actual calculation isn't implemented in the provided code snippet; it appears to be cut off.

In [10]:
_, _, time_travel_occured = simulation.check_simulation()

In [11]:
print(f"Did time travel occur? - {time_travel_occured}")

Did time travel occur? - False


### Results of the simulation

The percentage of cases meeting the threshol along with other informative statistics.  

In [12]:
simulation.analyze_simulation_results()

{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 100.0},
 'Mean Response Times': {'Immediate': 88.36074011402184,
  'Prompt': 84.70247039510707,
  'Standard': 88.49373834489559},
 'Mean Deployment Times': {'Immediate': 1.5403225806451613,
  'Prompt': 1.5061196105702366,
  'Standard': 1.4796755725190842},
 'Threshold Compliance': {'Immediate': 100.0,
  'Prompt': 99.58275382475661,
  'Standard': 99.23664122137404},
 'Mean Officer Hours': 56.9543844529505,
 'Unresolved Incident Percentage': 0.0}